In [0]:
!pip install lime
!pip install pandas
!pip install scikit-learn
!pip install pickle-mixin

In [0]:
import sklearn
import lime.lime_tabular
import pickle
import pandas as pd
import numpy as np

def unique(list1): 
    x = np.array(list1) 
    print(np.unique(x)) 

FEATURES_TO_OBSERVE = 10
filename = '/content/drive/My Drive/Predictions/Age_20_model.pickle'
rf_model = pickle.load(open(filename, 'rb'))

train_x = pd.read_pickle('/content/drive/My Drive/Predictions/Age_20_x_train.pickle')
train_y = pd.read_pickle('/content/drive/My Drive/Predictions/Age_20_y_train.pickle')
test_x = pd.read_pickle('/content/drive/My Drive/Predictions/Age_20_x_test.pickle')
test_y = pd.read_pickle('/content/drive/My Drive/Predictions/Age_20_y_test.pickle')

unique(test_y)

predict_fn_rf = lambda x: rf_model.predict_proba(x).astype(float)

print(len(train_x.index.values))
l = []
#create explainer object
explainer = lime.lime_tabular.LimeTabularExplainer(train_x.values,feature_names = train_x.columns,class_names=['20','40','60','80'],kernel_width=5)

c = 0
indexes = test_x.index.values
for i in indexes[0:1000]:
  print(c)
  choosen_instance = test_x.loc[[i]].values[0]
  exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=FEATURES_TO_OBSERVE)
  a = exp.as_list()
  l.append(a)
  print(i)
  c += 1

pickle.dump( l, open( "/content/drive/My Drive/Predictions/predictionAge.p", "wb" ) )

In [0]:
import pickle 

filename = '/content/drive/My Drive/Predictions/predictionAge.p'
predictions = pickle.load(open(filename, 'rb'))

print(predictions)

def process_predictions(predictions):
  data_elements = []
  for i in range(len(predictions)):
    for j in range(len(predictions[i])):

      index = predictions[i][j][0].find('f')
      space = predictions[i][j][0][index:index+5].find(' ')
      data_elements.append(predictions[i][j][0][index:(index + space)])    

  return data_elements

pred  = process_predictions(predictions)


In [0]:
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

#https://stackoverflow.com/questions/3594514/how-to-find-most-common-elements-of-a-list/18587325

def visualize_results(results):
  x = []
  y = []
  most_common = [item for item in Counter(results).most_common()]
  for i in range(len(most_common)):
    x.append(most_common[i][0])
    y.append(most_common[i][1])

  plt.bar(x, y, color='red')
  plt.xlabel("Features")
  plt.ylabel("Count")
  plt.title("Feature distribution")
  plt.show()
  print('Most common features:', most_common[0][0],most_common[1][0],most_common[2][0])

visualize_results(pred)